In [8]:
%pip install flask flask_cors markdown


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import sqlite3
from flask import Flask, request, jsonify
from flask_cors import CORS
import markdown
from threading import Thread

# -----------------------------
# Setup
# -----------------------------
app = Flask(__name__)
CORS(app)

DB_FILE = "notes.db"

# -----------------------------
# Simple Grammar Checker (No Java Needed)
# -----------------------------
def simple_grammar_check(text):
    issues = []

    # Rule 1: Double spaces
    if "  " in text:
        issues.append({
            "message": "Double space detected",
            "suggestions": [" "],
            "offset": text.index("  "),
            "errorLength": 2
        })

    # Rule 2: Common subject-verb error
    if "This are" in text:
        issues.append({
            "message": "Incorrect verb form",
            "suggestions": ["This is"],
            "offset": text.index("This are"),
            "errorLength": 7
        })

    # Rule 3: Common typos
    common_typos = {"teh": "the", "recieve": "receive", "adress": "address"}
    for typo, correction in common_typos.items():
        if typo in text:
            issues.append({
                "message": f"Possible typo: '{typo}'",
                "suggestions": [correction],
                "offset": text.index(typo),
                "errorLength": len(typo)
            })

    return {"corrections": issues}

# -----------------------------
# Initialize DB
# -----------------------------
def init_db():
    with sqlite3.connect(DB_FILE) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS notes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            content TEXT
        )
        """)
        conn.commit()

init_db()

# -----------------------------
# Endpoints
# -----------------------------

# 1. Grammar Check
@app.route("/check_grammar", methods=["POST"])
def check_grammar():
    data = request.get_json()
    text = data.get("text", "")
    corrections = simple_grammar_check(text)["corrections"]
    return jsonify({"corrections": corrections})

# 2. Save Note
@app.route("/save_note", methods=["POST"])
def save_note():
    data = request.get_json()
    title = data.get("title", "Untitled")
    content = data.get("content", "")
    with sqlite3.connect(DB_FILE) as conn:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO notes (title, content) VALUES (?, ?)", (title, content))
        conn.commit()
    return jsonify({"message": "Note saved successfully!"})

# 3. List Notes
@app.route("/list_notes", methods=["GET"])
def list_notes():
    with sqlite3.connect(DB_FILE) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT id, title FROM notes")
        notes = [{"id": row[0], "title": row[1]} for row in cursor.fetchall()]
    return jsonify(notes)

# 4. Render Markdown to HTML
@app.route("/render_note/<int:note_id>", methods=["GET"])
def render_note(note_id):
    with sqlite3.connect(DB_FILE) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT content FROM notes WHERE id=?", (note_id,))
        row = cursor.fetchone()
    if row:
        html_content = markdown.markdown(row[0])
        return jsonify({"html": html_content})
    else:
        return jsonify({"error": "Note not found"}), 404

# -----------------------------
# Run Flask in Background
# -----------------------------
def run_app():
    app.run(port=5000, debug=False)

thread = Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Aug/2025 19:05:49] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Aug/2025 19:05:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Aug/2025 19:06:21] "POST /check_grammar HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2025 19:06:29] "POST /save_note HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2025 19:06:38] "GET /list_notes HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2025 19:06:46] "GET /render_note/test_note HTTP/1.1" 404 -
127.0.0.1 - - [24/Aug/2025 19:06:50] "GET / HTTP/1.1" 404 -


In [10]:
import requests

# Test grammar check
res = requests.post("http://127.0.0.1:5000/check_grammar", json={
    "text": "This is a simple example with a mistake in it is text."
})
print(res.json())


{'corrections': []}


In [11]:
# Save a note
res = requests.post("http://127.0.0.1:5000/save_note", json={
    "title": "test_note",
    "content": "# Hello\nThis is my *first* note."
})
print(res.json())


{'message': 'Note saved successfully!'}


In [12]:
# List saved notes
res = requests.get("http://127.0.0.1:5000/list_notes")
print(res.json())


[{'id': 1, 'title': 'test_note'}]


In [13]:
# Render note into HTML
res = requests.get("http://127.0.0.1:5000/render_note/test_note")
print(res.text)   # This will be HTML


<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

